In [1]:
import sys
sys.path.append('../')
sys.path.append('../tusimple-benchmark/')
import json
import glob
import pickle

import numpy as np
import pandas as pd
import cv2
import joblib
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline

from evaluate.lane import LaneEval


map_json_gt = {json.loads(line)['raw_file']:json.loads(line) for line in open('../tusimple-test/test_label.json')}

# Benign Accuracy and F1 score

In [2]:
df_res = []
for model_type in ['laneatt', 'scnn', 'ultrafast', 'polylanenet']:
    print(model_type)
    paths = glob.glob(f'../logs/tusimple_benign/logs_{model_type}_replay/tusimple_*/replay/model_benign_lane_pred.pkl')
    res = []
    for path in tqdm(paths, desc=model_type):
        tmp = path.split('/')[-3].split('_')
        date = tmp[1]
        tt = tmp[2]
        try:
            gt = map_json_gt[f'clips/{date}/{tt}_0/20.jpg']
        except:
            gt = map_json_gt[f'clips/{date}/{tt}/20.jpg']


        run_time = 0 #pred['run_time']
        gt_lanes = gt['lanes']
        y_samples = gt['h_samples']
        raw_file = '../tusimple-test/' + gt['raw_file']

        #gt_lanes = sorted(gt_lanes, key=lambda x: (np.array(x) > 0).sum())[-2:]
        #gt_lanes = sorted(gt_lanes, key=lambda x: (np.array(x) > 0).mean())
        idx = np.array([np.array(x)[np.array(x) > 0].mean() for x in gt_lanes])
        right_i = np.argmin(np.where(idx > 640, idx - 640, np.inf))
        left_i = np.argmin(np.where(idx < 640, 640 - idx, np.inf))
        gt_lanes = [gt_lanes[left_i], gt_lanes[right_i]]

        with open(path, 'rb') as f:
            lanes = pickle.load(f)#joblib.load(path)
        left_lane, right_lane = lanes[-1]

        left_lane[:, 0] = (left_lane[:, 0] - 106) / 952 * 1280
        left_lane[:, 1] = (left_lane[:, 1] + 390 - 206) / 454 * 720
        right_lane[:, 0] = (right_lane[:, 0] - 106) / 952 * 1280
        right_lane[:, 1] = (right_lane[:, 1] + 390 - 206) / 454 * 720

        idx = np.argsort(left_lane[:, 1])
        left_lane = left_lane[idx]
        idx = np.argsort(right_lane[:, 1])
        right_lane = right_lane[idx]

        left_lane_x = InterpolatedUnivariateSpline(left_lane[:, 1], left_lane[:, 0], k=min(3, len(left_lane) - 1))(y_samples)
        right_lane_x = InterpolatedUnivariateSpline(right_lane[:, 1], right_lane[:, 0], k=min(3, len(right_lane) - 1))(y_samples)

        left_lane_x[np.array(gt_lanes[0]) == -2] = -2
        right_lane_x[np.array(gt_lanes[1]) == -2] = -2

        pred_lanes = [left_lane_x, right_lane_x]

        #np.random.shuffle(pred_lanes)
        # Overall Accuracy, False Positive Rate, False Negative Rate
        acc, _fp, _fn = LaneEval.bench(pred_lanes, gt_lanes, y_samples, run_time)
        
        fp = _fp * 2
        tp = 2 - fp 
        fn = _fn * 2
        f1 = tp / (tp + (fn + (2 - tp)) * 0.5)
        res.append([acc, f1])
        #import pdb;pdb.set_trace()
        
        #if sc[0] < 0.5:
        #    print(path, sc)
        #    #break
    print(len(res))
    tmp = pd.DataFrame(res, columns=['ACC', 'F1']).mean(axis=0)
    tmp['model_type'] = model_type
    df_res.append(tmp)
pd.concat(df_res, axis=1).T.set_index('model_type')

laneatt



30
scnn



30
ultrafast



30
polylanenet



30


,ACC,F1
model_type,,
laneatt,0.940476,0.883333
scnn,0.89256,0.75
ultrafast,0.867857,0.766667
polylanenet,0.721429,0.5


# Attacked Accuracy and F1 score

In [3]:
BASE_DIR = '../logs/tusimple_attack/'

df_res = []
for model_type in ['laneatt', 'scnn', 'ultrafast', 'polylanenet']:
    print(model_type)
    paths = glob.glob(BASE_DIR + f'logs_{model_type}_drp_wb/tusimple_*/replay/model_lane_pred.pkl')
    
    res = []
    for path in tqdm(paths, desc=model_type):
        tmp = path.split('/')[-3].split('_')
        date = tmp[1]
        tt = tmp[2]
        try:
            gt = map_json_gt[f'clips/{date}/{tt}_0/20.jpg']
        except:
            gt = map_json_gt[f'clips/{date}/{tt}/20.jpg']


        run_time = 0 #pred['run_time']
        gt_lanes = gt['lanes']
        y_samples = gt['h_samples']
        raw_file = '../tusimple-test/' + gt['raw_file']

        #gt_lanes = sorted(gt_lanes, key=lambda x: (np.array(x) > 0).sum())[-2:]
        #gt_lanes = sorted(gt_lanes, key=lambda x: (np.array(x) > 0).mean())
        idx = np.array([np.array(x)[np.array(x) > 0].mean() for x in gt_lanes])
        right_i = np.argmin(np.where(idx > 640, idx - 640, np.inf))
        left_i = np.argmin(np.where(idx < 640, 640 - idx, np.inf))
        gt_lanes = [gt_lanes[left_i], gt_lanes[right_i]]

        with open(path, 'rb') as f:
            lanes = pickle.load(f)#joblib.load(path)

        left_lane, right_lane = lanes[-1]

        left_lane[:, 0] = (left_lane[:, 0] - 106) / 952 * 1280
        left_lane[:, 1] = (left_lane[:, 1] + 390 - 206) / 454 * 720
        right_lane[:, 0] = (right_lane[:, 0] - 106) / 952 * 1280
        right_lane[:, 1] = (right_lane[:, 1] + 390 - 206) / 454 * 720

        idx = np.argsort(left_lane[:, 1])
        left_lane = left_lane[idx]
        idx = np.argsort(right_lane[:, 1])
        right_lane = right_lane[idx]

        left_lane_x = InterpolatedUnivariateSpline(left_lane[:, 1], left_lane[:, 0], k=min(3, len(left_lane) - 1))(y_samples)
        right_lane_x = InterpolatedUnivariateSpline(right_lane[:, 1], right_lane[:, 0], k=min(3, len(right_lane) - 1))(y_samples)

        left_lane_x[np.array(gt_lanes[0]) == -2] = -2
        right_lane_x[np.array(gt_lanes[1]) == -2] = -2

        pred_lanes = [left_lane_x, right_lane_x]

        #np.random.shuffle(pred_lanes)
        # Overall Accuracy, False Positive Rate, False Negative Rate
        acc, _fp, _fn = LaneEval.bench(pred_lanes, gt_lanes, y_samples, run_time)
        
        fp = _fp * 2
        tp = 2 - fp 
        fn = _fn * 2
        f1 = tp / (tp + (fn + (2 - tp)) * 0.5)
        res.append([acc, f1])
        #import pdb;pdb.set_trace()
        
        #if sc[0] < 0.5:
        #    print(path, sc)
        #    #break
    print(len(res))
    tmp = pd.DataFrame(res, columns=['ACC', 'F1']).mean(axis=0)
    tmp['model_type'] = model_type
    df_res.append(tmp)
pd.concat(df_res, axis=1).T.set_index('model_type')

laneatt



60
scnn



60
ultrafast



60
polylanenet



60


,ACC,F1
model_type,,
laneatt,0.510565,0.291667
scnn,0.580506,0.275
ultrafast,0.355208,0.075
polylanenet,0.531548,0.191667
